# Using Python for visualizing neuroimaging data

The primary goal of this section is to become familiar with loading, modifying, saving, and visualizing neuroimages in Python. A secondary goal is to develop a conceptual understanding of the data structures involved, to facilitate diagnosing problems in data or analysis pipelines.

To these ends, we'll be exploring two libraries: [nibabel](http://nipy.org/nibabel/) and [nilearn](https://nilearn.github.io/). Each of these projects has excellent documentation. While this should get you started, it is well worth your time to look through these sites.

In this notebook we will cover: [nilearn](https://nilearn.github.io/)

# Nilearn

[Nilearn](http://nilearn.github.io/index.html) labels itself as: *A Python module for fast and easy statistical learning on NeuroImaging data. It leverages the scikit-learn Python toolbox for multivariate statistics with applications such as predictive modelling, classification, decoding, or connectivity analysis.*

But it's much more than that. It is also an excelent library to **manipulate** (e.g. resample images, smooth images, ROI extraction, etc.) and **visulaize** your neuroimages.

So let's visit all three of those domains:

1. Image manipulation
2. Image visualization
3. Machine Learning

## Setup

In [ ]:
# Image settings
import pylab as plt
%matplotlib inline

import os
import numpy as np

out_dir = '/tmp'

## 1. Image manipulation with `nilearn`

### Let's create a mean image

Let's revisit the example from above and see how nilearn can create a **mean image** in just one line.

In [ ]:
from nilearn import image as nli

In [ ]:
img = nli.mean_img('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')

Perfect! What else can we do with the `image` module? Let's see:

In [ ]:
sorted([f for f in nli.__dict__ if '__' not in f])

### Resample image to a template
So, let's try **`resample_to_img`**:

In [ ]:
mean = nli.mean_img('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')
t1 = nli.load_img('/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')
print([mean.shape, t1.shape])

In [ ]:
# Let's resample the t1 image to the mean image
resampled_t1 = nli.resample_to_img(t1, mean)
resampled_t1.shape

The image size of the resampled t1 image seems to right, but how does it look like?

In [ ]:
from nilearn import plotting
plotting.plot_anat(t1, title='original t1', display_mode='z')
plotting.plot_anat(resampled_t1, title='resampled t1', display_mode='z')

### Smooth an image
Cool! What about **`smooth_img`**?

In [ ]:
for fwhm in range(0, 11, 5):
    smoothed_img = nli.smooth_img(mean, fwhm)
    plotting.plot_epi(smoothed_img, title="Smoothing %imm" % fwhm,
                     display_mode='z', cmap=plt.cm.magma)

### Clean an image to improve SNR

As simple as that. Before we continue, let's also look at **`clean_img`**. A function to improve the SNR of your fMRI signals. This can be done with one or more of the following options:

- detrend
- standardize
- remove confounds
- low- and high-pass filter

Low-pass filtering improves specificity. High-pass filtering should be kept small, to keep some sensitivity.

In [ ]:
# First, let's load again a functional image
func = nli.load_img('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii.gz')
TR = func.header['pixdim'][4]

In [ ]:
# As a first step, let's just detrend the image
func_d = nli.clean_img(func, detrend=True, standardize=False, t_r=TR)

In [ ]:
# Plot the original and detrended timecourse of a random voxel
x, y, z = [31, 14, 7]
plt.figure(figsize=(12, 4))
plt.plot(np.transpose(func.get_data()[x, y, z, :]))
plt.plot(np.transpose(func_d.get_data()[x, y, z, :]))
plt.legend(['Original', 'Detrend']);

Let's now see what `standardiz` does:

In [ ]:
func_ds = nli.clean_img(func, detrend=True, standardize=True, t_r=TR)

plt.figure(figsize=(12, 4))
plt.plot(np.transpose(func_d.get_data()[x, y, z, :]))
plt.plot(np.transpose(func_ds.get_data()[x, y, z, :]))
plt.legend(['Detrend', 'Detrend+standardize']);

In [ ]:
func_ds_c = nli.clean_img(func, detrend=True, standardize=True, t_r=TR,
                          confounds='data/sub-01_ses-test_task-fingerfootlips_bold_mcf.par')

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(np.transpose(func_ds.get_data()[x, y, z, :]))
plt.plot(np.transpose(func_ds_c.get_data()[x, y, z, :]))
plt.legend(['Det.+stand.', 'Det.+stand.-confounds']);

## 2. Image visualization with `nilearn`

In [ ]:
%matplotlib inline


Glass brain plotting in nilearn
===============================

See `plotting` for more plotting functionalities.



Retrieve data from Internet
---------------------------



In [ ]:
from nilearn import datasets

localizer_dataset = datasets.fetch_localizer_button_task()
localizer_tmap_filename = localizer_dataset.tmaps[0]

Glass brain plotting: whole brain sagittal cuts
-----------------------------------------------



In [ ]:
from nilearn import plotting

plotting.plot_glass_brain(localizer_tmap_filename, threshold=3)

Glass brain plotting: black backgrond
-------------------------------------
On a black background (option "black_bg"), and with only the x and
the z view (option "display_mode").



In [ ]:
plotting.plot_glass_brain(
    localizer_tmap_filename, title='plot_glass_brain',
    black_bg=True, display_mode='xz', threshold=3)

Glass brain plotting: Hemispheric sagittal cuts
-----------------------------------------------



In [ ]:
plotting.plot_glass_brain(localizer_tmap_filename,
                          title='plot_glass_brain with display_mode="lyrz"',
                          display_mode='lyrz', threshold=3)

plotting.show()


Glass brain plotting in nilearn (all options)
=============================================

First part of this example goes through different options of the
:func:`nilearn.plotting.plot_glass_brain` function (including plotting
negative values).

Second part, goes through same options but selected of the same glass brain
function but plotting is seen with contours.

See `plotting` for more plotting functionalities and
`Section 4.3 <display_modules>` for more details about display objects
in Nilearn.

Also, see :func:`nilearn.datasets.fetch_localizer_button_task` for details
about the plotting data and its experiments.



Retrieve the data
------------------

Nilearn comes with set of functions that download public data from Internet

Let us first see where the data will be downloded and stored on our disk:




In [ ]:
from nilearn import datasets
print('Datasets shipped with nilearn are stored in: %r' % datasets.get_data_dirs())

Let us now retrieve a motor task contrast maps corresponding to second subject
from a localizer experiment



In [ ]:
tmap_filenames = datasets.fetch_localizer_button_task()['tmaps']
print(tmap_filenames)

tmap_filenames is returned as a list. We need to take first one



In [ ]:
tmap_filename = tmap_filenames[0]

Demo glass brain plotting
--------------------------



In [ ]:
from nilearn import plotting

# Whole brain sagittal cuts and map is thresholded at 3
plotting.plot_glass_brain(tmap_filename, threshold=3)

With a colorbar



In [ ]:
plotting.plot_glass_brain(tmap_filename, threshold=3, colorbar=True)

Black background, and only the (x, z) cuts



In [ ]:
plotting.plot_glass_brain(tmap_filename, title='plot_glass_brain',
                          black_bg=True, display_mode='xz', threshold=3)

Plotting the sign of the activation with plot_abs to False



In [ ]:
plotting.plot_glass_brain(tmap_filename, threshold=0, colorbar=True,
                          plot_abs=False)

The sign of the activation and a colorbar



In [ ]:
plotting.plot_glass_brain(tmap_filename, threshold=3,
                          colorbar=True, plot_abs=False)

Different projections for the left and right hemispheres
---------------------------------------------------------

Hemispheric sagittal cuts



In [ ]:
plotting.plot_glass_brain(tmap_filename,
                          title='plot_glass_brain with display_mode="lzr"',
                          black_bg=True, display_mode='lzr', threshold=3)

In [ ]:
plotting.plot_glass_brain(tmap_filename, threshold=0, colorbar=True,
                          title='plot_glass_brain with display_mode="lyrz"',
                          plot_abs=False, display_mode='lyrz')

Demo glass brain plotting with contours and with fillings
---------------------------------------------------------
To plot maps with contours, we call the plotting function into variable from
which we can use specific display features which are inherited automatically.
In this case, we focus on using add_contours
First, we initialize the plotting function into "display" and first
argument set to None since we want an empty glass brain to plotting the
statistical maps with "add_contours"



In [ ]:
display = plotting.plot_glass_brain(None)
# Here, we project statistical maps
display.add_contours(tmap_filename)
# and a title
display.title('"tmap_filename" on glass brain without threshold')

Plotting with `filled=True` implies contours with fillings. Here, we are not
specifying levels



In [ ]:
display = plotting.plot_glass_brain(None)
# Here, we project statistical maps with filled=True
display.add_contours(tmap_filename, filled=True)
# and a title
display.title('Same map but with fillings in the contours')

Here, we input specific level (cut-off) in the statistical map. In other way,
we are thresholding our statistical map



In [ ]:
# Here, we set the threshold using parameter called `levels` with value given
# in a list and choosing color to Red.
display = plotting.plot_glass_brain(None)
display.add_contours(tmap_filename, levels=[3.], colors='r')
display.title('"tmap_filename" on glass brain with threshold')

Plotting with same demonstration but inlcudes now filled=True



In [ ]:
display = plotting.plot_glass_brain(None)
display.add_contours(tmap_filename, filled=True, levels=[3.], colors='r')
display.title('Same demonstration but using fillings inside contours')

Plotting with black background, `black_bg` should be set with
`plot_glass_brain`



In [ ]:
# We can set black background using black_bg=True
display = plotting.plot_glass_brain(None, black_bg=True)
display.add_contours(tmap_filename, levels=[3.], colors='g')
display.title('"tmap_filename" on glass brain with black background')

Black background plotting with filled in contours



In [ ]:
display = plotting.plot_glass_brain(None, black_bg=True)
display.add_contours(tmap_filename, filled=True, levels=[3.], colors='g')
display.title('Glass brain with black background and filled in contours')

Display contour projections in both hemispheres
-------------------------------------------------
Key argument to vary here is `display_mode` for hemispheric plotting



In [ ]:
# Now, display_mode is chosen as 'lr' for both hemispheric plots
display = plotting.plot_glass_brain(None, display_mode='lr')
display.add_contours(tmap_filename, levels=[3.], colors='r')
display.title('"tmap_filename" on glass brain only "l" "r" hemispheres')

Filled contours in both hemispheric plotting, just by adding filled=True



In [ ]:
display = plotting.plot_glass_brain(None, display_mode='lr')
display.add_contours(tmap_filename, filled=True, levels=[3.], colors='r')
display.title('Filled contours on glass brain only "l" "r" hemispheres')

With positive and negative sign of activations with `plot_abs` in
`plot_glass_brain`



In [ ]:
# By default parameter `plot_abs` is True and sign of activations can be
# displayed by changing `plot_abs` to False
display = plotting.plot_glass_brain(None, plot_abs=False, display_mode='lzry')
display.add_contours(tmap_filename)
display.title("Contours with both sign of activations without threshold")

Now, adding just filled=True to get positive and negative sign activations
with fillings in the contours



In [ ]:
display = plotting.plot_glass_brain(None, plot_abs=False, display_mode='lzry')
display.add_contours(tmap_filename, filled=True)
display.title("Filled contours with both sign of activations without threshold")

Displaying both signs (positive and negative) of activations with threshold
meaning thresholding by adding an argument `levels` in add_contours.



In [ ]:
import numpy as np
display = plotting.plot_glass_brain(None, plot_abs=False, display_mode='lzry')

# In add_contours,
# we give two values through the argument `levels` which corresponds to the
# thresholds of the contour we want to draw: One is positive and the other one
# is negative. We give a list of `colors` as argument to associate a different
# color to each contour. Additionally, we also choose to plot contours with
# thick line widths, For linewidths one value would be enough so that same
# value is used for both contours.
display.add_contours(tmap_filename, levels=[-2.8, 3.], colors=['b', 'r'],
                     linewidths=4.)
display.title('Contours with sign of activations with threshold')

Same display demonstration as above but just adding filled=True to get
fillings inside the contours.



In [ ]:
# Unlike in previous plot, here we specify each sign at a time. We call negative
# values display first followed by positive values display.

# First, we fetch our display object with same parametes used as above
display = plotting.plot_glass_brain(None, plot_abs=False, display_mode='lzry')

# Second, we plot negative sign of activation with levels given as negative
# activation value in a list. Upper bound should be kept to -infinity
display.add_contours(tmap_filename, filled=True, levels=[-np.inf, -2.8],
                     colors='b')
# Next, within same plotting object we plot positive sign of activation
display.add_contours(tmap_filename, filled=True, levels=[3.], colors='r')
display.title('Now same plotting but with filled contours')

# Finally, displaying them
plotting.show()


More plotting tools from nilearn
================================

In this example, we demonstrate how to use plotting options from
nilearn essential in visualizing brain image analysis results.

We emphasize the use of parameters such as `display_mode` and `cut_coords`
with plotting function :func:`nilearn.plotting.plot_stat_map`. Also,
we show how to use various features such as `add_edges`, `add_contours`,
`add_markers` essential in visualizing regions of interest images or
mask images overlaying on subject specific anatomical/EPI image.
The display features shown here are inherited from the
:class:`nilearn.plotting.displays.OrthoSlicer` class.

The parameter `display_mode` is used to draw brain slices along given
specific directions, where directions can be one of 'ortho',
'x', 'y', 'z', 'xy', 'xz', 'yz'. whereas parameter `cut_coords`
is used to specify a limited number of slices to visualize along given
specific slice direction. The parameter `cut_coords` can also be used
to draw the specific cuts in the slices by giving its particular
coordinates in MNI space accordingly with particular slice direction.
This helps us point to the activation specific location of the brain slices.

See `plotting` for more details.



First, we retrieve data from nilearn provided (general-purpose) datasets
-------------------------------------------------------------------------



In [ ]:
from nilearn import datasets

# haxby dataset to have anatomical image, EPI images and masks
haxby_dataset = datasets.fetch_haxby()
haxby_anat_filename = haxby_dataset.anat[0]
haxby_mask_filename = haxby_dataset.mask_vt[0]
haxby_func_filename = haxby_dataset.func[0]

# localizer dataset to have contrast maps
localizer_dataset = datasets.fetch_localizer_button_task(get_anats=True)
localizer_anat_filename = localizer_dataset.anats[0]
localizer_tmap_filename = localizer_dataset.tmaps[0]

Now, we show from here how to visualize the retrieved datasets using plotting
tools from nilearn.



In [ ]:
from nilearn import plotting

Visualizing in - 'sagittal', 'coronal' and 'axial' with given coordinates
-------------------------------------------------------------------------
The first argument is a path to the filename of a constrast map,
optional argument `display_mode` is given as string 'ortho' to visualize
the map in three specific directions xyz and the optional `cut_coords`
argument, is here a list of integers denotes coordinates of each slice
in the order [x, y, z]. By default the `colorbar` argument is set to True
in plot_stat_map.



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='ortho',
                       cut_coords=[36, -27, 60],
                       title="display_mode='ortho', cut_coords=[36, -27, 60]")

Visualizing in - single view 'axial' with number of cuts=5
-----------------------------------------------------------
In this type of visualization, the `display_mode` argument is given as
string 'z' for axial direction and `cut_coords` as integer 5 without a
list implies that number of cuts in the slices should be maximum of 5.
The coordinates to cut the slices are selected automatically



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='z', cut_coords=5,
                       title="display_mode='z', cut_coords=5")

Visualizing in - single view 'sagittal' with only two slices
-------------------------------------------------------------
In this type, `display_mode` should be given as string 'x' for sagittal
view and coordinates should be given as integers in a list



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='x',
                       cut_coords=[-36, 36],
                       title="display_mode='x', cut_coords=[-36, 36]")

Visualizing in - 'coronal' view with single cut
------------------------------------------------
For coronal view, `display_mode` is given as string 'y' and `cut_coords`
as integer 1 not as a list for single cut. The coordinates are selected
automatically



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='y', cut_coords=1,
                       title="display_mode='y', cut_coords=1")

Visualizing without a colorbar on the right side
-------------------------------------------------
The argument `colorbar` should be given as False to show plots without
a colorbar on the right side.



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='z',
                       cut_coords=1, colorbar=False,
                       title="display_mode='z', cut_coords=1, colorbar=False")

Visualize in - two views 'sagittal' and 'axial' with given coordinates
-------------------------------------------------------------------------
argument display_mode='xz' where 'x' for sagittal and 'z' for axial view.
argument `cut_coords` should match with input number of views therefore two
integers should be given in a list to select the slices to be displayed



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='xz',
                       cut_coords=[36, 60],
                       title="display_mode='xz', cut_coords=[36, 60]")

Changing the views to 'coronal', 'sagittal' views with coordinates
-------------------------------------------------------------------
display_mode='yx' for coronal and saggital view and coordinates will be
assigned in the order of direction as [x, y, z]



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='yx',
                       cut_coords=[-27, 36],
                       title="display_mode='yx', cut_coords=[-27, 36]")

Now, views are changed to 'coronal' and 'axial' views with coordinates
-----------------------------------------------------------------------



In [ ]:
plotting.plot_stat_map(localizer_tmap_filename, display_mode='yz',
                       cut_coords=[-27, 60],
                       title="display_mode='yz', cut_coords=[-27, 60]")

Demonstrating various display features
---------------------------------------
In second part, we switch to demonstrating various features add_* from
nilearn where each specific feature will be helpful in projecting brain
imaging results for further interpretation.



In [ ]:
# Import image processing tool for basic processing of functional brain image
from nilearn import image

# Compute voxel-wise mean functional image across time dimension. Now we have
# functional image in 3D assigned in mean_haxby_img
mean_haxby_img = image.mean_img(haxby_func_filename)

Showing how to use `add_edges`
------------------------------
Now let us see how to use `add_edges`, method useful for checking
coregistration by overlaying anatomical image as edges (red) on top of
mean functional image (background), both being of same subject.



In [ ]:
# First, we call the `plot_anat` plotting function, with a background image
# as first argument, in this case the mean fMRI image.

display = plotting.plot_anat(mean_haxby_img, title="add_edges")

# We are now able to use add_edges method inherited in plotting object named as
# display. First argument - anatomical image  and by default edges will be
# displayed as red 'r', to choose different colors green 'g' and  blue 'b'.
display.add_edges(haxby_anat_filename)

How to use `add_contours`
-------------------------
Plotting outline of the mask (red) on top of the mean EPI image with
`add_contours`. This method is useful for region specific interpretation
of brain images



In [ ]:
# As seen before, we call the `plot_anat` function with a background image
# as first argument, in this case again the mean fMRI image and argument
# `cut_coords` as list for manual cut with coordinates pointing at masked
# brain regions
display = plotting.plot_anat(mean_haxby_img, title="add_contours",
                             cut_coords=[-34, -39, -9])
# Now use `add_contours` in display object with the path to a mask image from
# the Haxby dataset as first argument and argument `levels` given as list
# of values to select particular level in the contour to display and argument
# `colors` specified as red 'r' to see edges as red in color.
# See help on matplotlib.pyplot.contour to use more options with this method
display.add_contours(haxby_mask_filename, levels=[0.5], colors='r')

Plotting outline of the mask (blue) with color fillings using same method
`add_contours`.



In [ ]:
display = plotting.plot_anat(mean_haxby_img,
                             title="add_contours with filled=True",
                             cut_coords=[-34, -39, -9])

# By default, no color fillings will be shown using `add_contours`. To see
# contours with color fillings use argument filled=True. contour colors are
# changed to blue 'b' with alpha=0.7 sets the transparency of color fillings.
# See help on matplotlib.pyplot.contourf to use more options given that filled
# should be True
display.add_contours(haxby_mask_filename, filled=True, alpha=0.7,
                     levels=[0.5], colors='b')

Plotting seeds using `add_markers`
----------------------------------
Plotting seed regions of interest as spheres using new feature `add_markers`
with MNI coordinates of interest.



In [ ]:
display = plotting.plot_anat(mean_haxby_img, title="add_markers",
                             cut_coords=[-34, -39, -9])

# Coordinates of seed regions should be specified in first argument and second
# argument `marker_color` denotes color of the sphere in this case yellow 'y'
# and third argument `marker_size` denotes size of the sphere
coords = [(-34, -39, -9)]
display.add_markers(coords, marker_color='y', marker_size=100)

Finally, saving the plots to file with two different ways



In [ ]:
# Contrast maps plotted with function `plot_stat_map` can be saved using an
# inbuilt parameter output_file as filename + .extension as string. Valid
# extensions are .png, .pdf, .svg
plotting.plot_stat_map(localizer_tmap_filename,
                       title='Using plot_stat_map output_file',
                       output_file='plot_stat_map.png')

Another way of saving plots is using 'savefig' option from display object



In [ ]:
display = plotting.plot_stat_map(localizer_tmap_filename,
                                 title='Using display savefig')
display.savefig('plot_stat_map_from_display.png')
# In non-interactive settings make sure you close your displays
display.close()

plotting.show()


Making a surface plot of a 3D statistical map
=============================================

project a 3D statistical map onto a cortical mesh using
:func:`nilearn.surface.vol_to_surf`. Display a surface plot of the projected
map using :func:`nilearn.plotting.plot_surf_stat_map`.

NOTE: Example needs matplotlib version higher than 1.3.1.




Get a statistical map
---------------------



In [ ]:
from nilearn import datasets

localizer_dataset = datasets.fetch_localizer_button_task()
localizer_tmap = localizer_dataset.tmaps[0]

Get a cortical mesh
-------------------



In [ ]:
fsaverage = datasets.fetch_surf_fsaverage5()

Sample the 3D data around each node of the mesh
-----------------------------------------------



In [ ]:
from nilearn import surface

texture = surface.vol_to_surf(localizer_tmap, fsaverage.pial_right)

Plot the result
---------------



In [ ]:
from nilearn import plotting

plotting.plot_surf_stat_map(fsaverage.infl_right, texture, hemi='right',
                            title='Surface right hemisphere',
                            threshold=1., bg_map=fsaverage.sulc_right,
                            cmap='cold_hot')

Plot 3D image for comparison
----------------------------



In [ ]:
plotting.plot_glass_brain(localizer_tmap, display_mode='r', plot_abs=False,
                          title='Glass brain', threshold=2.)

plotting.plot_stat_map(localizer_tmap, display_mode='x', threshold=1.,
                       cut_coords=range(0, 51, 10), title='Slices')

plotting.show()

# Don't forget tal's visualization
https://github.com/neurohackweek/visualization-in-python/blob/master/visualization-in-python.ipynb

## 3. Machine learning with `nilearn`


It also simplifies a number of common tasks with neuroimages. For example, we can recreate the mean EPI image we just made in one line:

In [ ]:
import nilearn as nl
import nilearn.plotting


In [ ]:
img = nl.image.mean_img('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-linebisection_bold.nii.gz')
print(img)

Nilearn images are just nibabel images! But notice that we didn't have to copy the affine or header. Nilearn does its best to keep your data and metadata together.

Let's verify that both methods produced the same image.

In [ ]:
np.array_equal(mean_epi.get_data(), img.get_data())

In [ ]:
np.array_equal(mean_epi.affine, img.affine)

## Plotting images

Nilearn has a variety of plotting facilities. `plot_epi` shows functional images in a high-contrast color scheme.

In [ ]:
nl.plotting.plot_epi(mean_epi, cut_coords=(0, 0, 0))

In [ ]:
help(nl.plotting.plot_epi)

## Exercise 4:

Using the help output from above, redraw the mean_epi image as a set of 5 slices spanning front to back. Suppress the background using the `vmin` option.

In [ ]:
nl.plotting.plot_epi(mean_epi, display_mode='y', cut_coords=5, vmin=10)

In [ ]:
# plot the mean_epi image slices

We can also plot anatomical images. Let's show the FreeSurfer `aseg` segmentation over the T1 image we loaded earlier:

In [ ]:
nl.plotting.plot_anat(t1, dim=-1, cut_coords=(0, 0, 0))
nl.plotting.plot_roi('/data/ds000114/derivatives/freesurfer/sub-01/mri/aseg.mgz', t1,
                     dim=-1, cut_coords=(0, 0, 0))

But the overlay above looks misaligned. That's because this dataset uses a derived T1 image as input to `FreeSurfer`.

## Exercise 5:

The T1 image used for FreeSurfer is at `/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz`. Redraw the above plot using the correct background T1 image.

In [ ]:
t1_correct = nib.load('/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz')
nl.plotting.plot_anat(t1_correct, dim=-1, cut_coords=(0, 0, 0))
nl.plotting.plot_roi('/data/ds000114/derivatives/freesurfer/sub-01/mri/aseg.mgz', t1_correct,
                     dim=-1, cut_coords=(0, 0, 0))

Notice that nilearn will accept an image or a filename equally. Also recall that `t1` was a NIfTI-1 image, while `aseg` is in a FreeSurfer `.mgz` file. Nilearn takes advantage of the common interface (data-affine-header) that nibabel provides for these different formats, and makes correctly aligned overlays.

This means we can use nilearn to verify alignment, for example when testing a new algorithm.

In [ ]:
def new_algorithm(image):
    # Just mess up the affine
    bad_affine = image.affine.copy()
    bad_affine[:, :2] = mask.affine[:, 1::-1]
    return image.__class__(image.get_data(), bad_affine, mask.header)

mask = nl.image.math_img("img > 0", img='/data/ds000114/derivatives/freesurfer/sub-01/mri/brainmask.auto.mgz')
new_mask = new_algorithm(mask)

## Exercise 6:

Plot the original and messed up mask on the same background image with two colors.

In [ ]:
t1_correct = nib.load('/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz')
nl.plotting.plot_roi(mask, t1_correct, dim=-1, cut_coords=(0, 0, 0), cmap='Greens_r')
nl.plotting.plot_roi(new_mask, t1_correct, dim=-1, cut_coords=(0, 0, 0), cmap='Reds_r')

Nilearn can also plot results directly in MNI space using an outline. This uses the function `nl.plotting.plot_glass_brain`

In [ ]:
img = nl.image.mean_img('/data/ds000114/derivatives/fmriprep/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold_space-mni152nlin2009casym_preproc.nii.gz')
nl.plotting.plot_glass_brain(img, threshold=1000, colorbar=True, display_mode='lyrz')

## Plotting surfaces

Nilearn has recently added surface plotting to its repertoire. Let's examine the gray/white boundary and pial surfaces.

In [ ]:
white = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.white'
pial = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.pial'
sulc = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.sulc'

In [ ]:
_ = nl.plotting.plot_surf(white, bg_map=sulc)
_ = nl.plotting.plot_surf(pial, bg_map=sulc)

A common step in surface pipelines is to create a surface halfway between the white and pial surface (often called the "midthickness" or "graymid" surface). The fastest, easiest, possibly wrong (but in practice fine) way to get this surface is to take the mean of the coordinates of the corresponding vertices on the white and pial surface. We can do this straightforwardly in nibabel.

In [ ]:
wcoords, wfaces, wmeta = nb.freesurfer.read_geometry(white, read_metadata=True)
pcoords, pfaces = nb.freesurfer.read_geometry(white, read_metadata=False)

# Make sure these surfaces actually do correspond
assert np.array_equal(wfaces, pfaces)

(wcoords, wfaces, wmeta)

Notice that this is not an image object, just a tuple of coordinates, faces, and an optional metadata dictionary. And it's an example of a file nibabel doesn't handle with `nibabel.load()`.

Coordinates are the (x, y, z) coordinates of each vertex; faces are a triangle composed of three vertices, and the metadata describes the provenance and alignment.

In [ ]:
gcoords = (wcoords + pcoords) / 2
# nilearn can be pretty picky about names, so fool it into reading this as a surface file
graymid = os.path.join(out_dir, 'lh.white')
nb.freesurfer.write_geometry(graymid, gcoords, wfaces, volume_info=wmeta)

In [ ]:
_ = nl.plotting.plot_surf(graymid, bg_map=sulc, view='lateral')
_ = nl.plotting.plot_surf(graymid, bg_map=sulc, view='medial')

Looks reasonable. Let's overlay it with the `aparc` parcellation. Nilearn doesn't handle these well yet, so again, we'll load with nibabel.

In [ ]:
aparc = nb.freesurfer.read_annot('/data/ds000114/derivatives/freesurfer/sub-01/label/lh.aparc.annot')

_ = nl.plotting.plot_surf_roi(os.path.join(out_dir, 'lh.white'), aparc[0], bg_map=sulc, view='lateral')

## Exercise 7:

Plot the aparc overlay of the right hemisphere of subject 2 after calculating the mid-thickness geometry.

In [ ]:
white = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.white'
pial = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.pial'
sulc = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.sulc'

wcoords, wfaces, wmeta = nb.freesurfer.read_geometry(white, read_metadata=True)
pcoords, pfaces = nb.freesurfer.read_geometry(white, read_metadata=False)

gcoords = (wcoords + pcoords) / 2
# nilearn can be pretty picky about names, so fool it into reading this as a surface file
graymid = os.path.join(out_dir, 'rh.white')
nb.freesurfer.write_geometry(graymid, gcoords, wfaces, volume_info=wmeta)

aparc = nb.freesurfer.read_annot('/data/ds000114/derivatives/freesurfer/sub-02/label/rh.aparc.annot')

_ = nl.plotting.plot_surf_roi(os.path.join(out_dir, 'rh.white'), aparc[0], bg_map=sulc, view='lateral')

In [ ]:
# Point to data

# Load data

# Calculate mid thickness

# Plot

### Review

We've explored the visualization capabilities of nilearn, which include the ability to plot BOLD images, ROIs and masks overlaid on anatomical images and surfaces. Additionally, we've used nilearn's image manipulation utilities (`mean_img`, and `math_img`) to quickly create new, valid images, and considered the dangers of destroying your affine matrix. Finally, we created our own surface using nibabel's FreeSurfer utilities.

## 3. Machine learning with `nilearn`

Although nilearn's visualizations are quite nice, its primary purpose was to make running machine learning algorithms on neuroimaging data as simple as possible. In this sense, it is a bridge between nibabel and [scikit-learn](http://scikit-learn.org/stable/). On the one hand, it reformats images to be easily passed to scikit-learn, and on the other, it reformats the results to produce valid nibabel images.

This section is heavily based on the [nilearn decoding tutorial](https://nilearn.github.io/auto_examples/plot_decoding_tutorial.html).

The dataset should be pre-loaded for you on your docker image. Go ahead and verify:

In [ ]:
import nilearn.datasets
import nilearn.input_data

haxby_dataset = nl.datasets.fetch_haxby(data_dir='data')

bold = haxby_dataset.func[0]
mask = haxby_dataset.mask_vt[0]
anat = haxby_dataset.anat[0]
labels = haxby_dataset.session_target[0]

!nib-ls $bold

### Masking and Un-masking data

We need our functional data in a 2D, sample-by-voxel matrix. To get that, we'll select a set of voxels in VT cortex defined by `mask`.

In [ ]:
nl.plotting.plot_roi(mask, anat, cmap='Paired', dim=-.5)

`NiftiMasker` is an object that applies a mask to a dataset and returns the masked voxels as a vector at each time point.
`standardize=True` z-scores each voxel.

In [ ]:
masker = nl.input_data.NiftiMasker(mask_img=mask, standardize=True)
samples = masker.fit_transform(bold)
print(samples)
print(samples.shape)

To recover the original data shape (giving us a masked and z-scored BOLD series), we simply use the masker's inverse transform:

In [ ]:
masked_epi = masker.inverse_transform(samples)

# For visualization... not a useful statistic
max_zscores = nl.image.math_img("np.abs(img).max(axis=3)", img=masked_epi)
nl.plotting.plot_stat_map(max_zscores, bg_img=anat, dim=-.5)

### Simple MVPA Example

Multi-voxel pattern analysis (MVPA) is a general term for techniques that contrast conditions over multiple voxels. It's very common to use machine learning models to generate statistics of interest.

In this case, we'll use the response patterns of voxels in VT cortex to predict the identity of the stimulus this subject was presented with. We'll use a support vector classifier (SVC) and leave-one-run-out cross-validation.

This section is not intended to teach machine learning, but to demonstrate a simple nilearn pipeline.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score

The labels file contains metadata for each volume, indicating the stimulus type and run number.

In [ ]:
!head -n 5 $labels

Using `np.recfromcsv()`, we can refer to each column of this file by its header.

In [ ]:
attrs = np.recfromcsv(labels, delimiter=" ")
stimuli, runs = attrs['labels'], attrs['chunks']
print(attrs.shape, np.unique(stimuli), np.unique(runs), sep='\n')

For simplicity, let's consider a two-class problem. Select the BOLD samples associated with bottles and shoes. We'll also need to select the corresponding stimuli and run numbers.

In [ ]:
condition_mask = (stimuli == b'bottle') | (stimuli == b'shoe')

samples_2class = samples[condition_mask]
stimuli_2class = stimuli[condition_mask]
runs_2class = runs[condition_mask]

samples_2class.shape

Leave-one-run-out cross-validation trains on `(n - 1)` runs, and classifies the remaining run, for each run. Mean (across runs) cross-validation accuracy is a common statistic for classification-based MVPA.

In [ ]:
svc = SVC(kernel='linear')
cva = cross_val_score(estimator=svc,
                      X=samples_2class,
                      y=stimuli_2class,
                      groups=runs_2class,
                      cv=LeaveOneGroupOut(),
                      n_jobs=-1)
print(cva, cva.mean(), sep='\n')

Another approach is to train a classifier on all of the data. This isn't useful for predicting, but we can read out the weight assigned to each voxel, giving a measure of its correlation with the stimulus type.

In [ ]:
svc.fit(samples_2class, stimuli_2class)
svc.coef_.shape

Since we have a value for each voxel, we can simply map this back to the volume using our `masker`, and visualize the weights.

In [ ]:
coef_vol = masker.inverse_transform(svc.coef_)
nl.plotting.plot_stat_map(coef_vol, bg_img=anat, dim=-.5)

### Review

In this section, we explored nilearn's tools for interfacing neuroimaging data and machine learning algorithms. Central to this is the concept of the masker, which moves data from 4-dimensional BOLD time series to a 2-dimensional series of feature vectors, and can map resulting statistics back into the original BOLD volume. We used leave-one-run-out cross-validation to explore 2-class support vector classification, and mapped feature weights back into the volume.

# Conclusions

In this tutorial, we've explored loading, saving and visualizing neuroimages, as well as how nibabel and nilearn can make some more sophisticated manipulations easy. At this point, you should be able to inspect and plot most images you encounter, as well as make modifications while preserving the alignment. If we've made it through the final section, you've also seen the basic workflow for performing a wide range of statistical analyses on BOLD time series in nilearn.